In [5]:
a = np.ones(3)
b = ["lol", "lol2", "lol3"]

str(dict(zip(b,a)))

"{'lol': 1.0, 'lol2': 1.0, 'lol3': 1.0}"

In [2]:
import pandas as pd
import numpy as np
import cv2
import torch
import matplotlib.pyplot as plt
import os
from skimage.io import imread, imsave
from skimage.color import rgb2gray, rgba2rgb, gray2rgb
import pytorch_lightning as pl

ds_root = '/home/alexander/work/hackathon/chest-14/images/'

df_only_labels = pd.read_csv('/home/alexander/work/hackathon/chest-14/Data_Entry_2017.csv')

from functools import reduce 
pathologies = sorted(list(reduce(lambda acc, str: set(str.split('|')).union(set(acc)),df_only_labels['Finding Labels'].unique(), set([]))))
del pathologies[-5]

from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self, csv, path_to_root, pathologies, aug = None):
        super(Dataset, self).__init__()
        
        self.path_to_root = path_to_root
        self.csv = csv
        self.pathologies = pathologies
        self.aug = aug
        
        self.labels = []
        for pathology in self.pathologies:
            self.labels.append(self.csv['Finding Labels'].str.contains(pathology).values)
            
        self.labels = np.asarray(self.labels).T
        self.labels = self.labels.astype(np.float32)
        
    def __getitem__(self, idx):
        item = self.csv.iloc[idx]
        
        img = imread(os.path.join(self.path_to_root,item['Image Index']))
        if img.ndim==2:
            img = gray2rgb(img)
        if img.shape[2]==4:
            img = img[:,:,:3]
        label = self.labels[idx]
        
        if self.aug:
            img = self.aug(image=img)['image']
            
        sample = {"image": img, 'label': label}
            
        return sample
    
    def __len__(self):
        return len(self.csv)
    
from albumentations.core.composition import *
from albumentations.pytorch import ToTensor, ToTensorV2
from albumentations.augmentations.transforms import *
    
tr = Compose([Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), Resize(224, 224), ToTensorV2()])

ds = MyDataset(df_only_labels, ds_root, pathologies = pathologies, aug = tr)

In [15]:
import torchvision.models as models

densenet = models.densenet121(pretrained=True)


In [16]:
densenet.classifier = torch.nn.Identity(1024)

In [18]:
a = densenet(ds[0]['image'])

In [23]:
processed = []

for im in ds:
    pred = densenet(im['image'].unsqueeze(0))
    processed.append({
            "features": pred[0,:],
            "labels": im['label']
        })
    break
    
processed

[{'features': tensor([3.5344e-04, 4.3502e-03, 3.1985e-03,  ..., 1.4926e+00, 1.0923e+00,
          1.3415e+00], grad_fn=<SliceBackward>),
  'labels': array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        dtype=float32)}]